`splatlog.setup` Notes
==============================================================================

`splatlog.setup` is the function you need to call to get any logging output,
which is typically what you're looking for when you're throwing something
together quickly or jamming `splatlog` in because you need some help seeing
what's going on.

To this end, the goal of `splatlog.setup` is to be "reference-free", in the
sense that you don't need to look anything up to get what you want out of it.
That means that, ideally, whatever makes sense to write in a `setup` call should
_just work_ if reasonably possible.

That aside, `setup` should be documented, and it should be documented well. At
the moment, that's not the case.

This notebook is dedicated to:

1.  Fiddling around when writing the `splatlog.setup` doc.
2.  Exploring and cataloging the ways it "makes sense" to me to call `setup`,
    to see if we can support them.

In [1]:
import splatlog
from collections.abc import Mapping
from typing import Any
from rich.console import Console
import logging
import sys
import rich.logging
from io import StringIO
import json
from rich.text import Text
from IPython.display import HTML, display
import re

# Before anything else fix the bad ANSI blue
splatlog.setup(
    theme=splatlog.rich.override_ansi_colors(
        black="#0e0f12",
        red="#e06c75",
        bright_green="#5cb85c",
        bright_yellow="#f0ad4e",
        bright_blue="#52acf7",
        bright_magenta="#b95bde",
        bright_cyan="#5bc0de",
        bright_white="#ffffff",
        green="#98c379",
        yellow="#e5c07b",
        blue="#61afef",
        magenta="#c678dd",
        cyan="#56b6c2",
        white="#dee1de",
        bright_black="#636a80",
        bright_red="#d9534f",
    )
)

LOG = splatlog.getLogger(__name__)

The `level` argument sets the root or master level, which is the level of the
root/master logger returned by `logging.getLogger()` or
`logging.getLogger("root")`.

The default level before anything is done is `WARNING`, which comes from the 
`logging` module.

`splatlog` provides `get_level()`, `set_level(level)`, and `get_level_name()`
helpers.

In [2]:
assert splatlog.get_level() == splatlog.WARNING
splatlog.get_level_name()

'WARNING'

In the beginning... there are no handlers. If you log at `WARNING` or above,
you'll hit `logging.lastResort`, which will print to `STDERR`. Nice to know, but
not that interesting.

In [3]:
assert len(list(LOG.iter_handlers())) == 0
LOG.warning("Appears on STDERR")

Appears on STDERR


The typical, basic, "let me see something" setup consists of a `level` and
`console` handler.

In [4]:
splatlog.setup(level="info", console=True)

LOG.info("Hey!")

INFO      __main__                                                                                                 
msg       Hey!

The default output is `STDERR`, same as `logging.lastResort`.

In [5]:
h = splatlog.get_named_handler("console")
assert isinstance(h, splatlog.RichHandler)
assert h.console.file == sys.stderr
assert h.console.file != sys.stdout

The named handlers can be constructed from a level, but the root `level` will
dominate, so it only really makes sense with multiple handlers.

In [6]:
splatlog.setup(level="info", console="debug")

# Does _not_ show
LOG.debug("Shown?")

ch = splatlog.get_named_handler("console")
assert isinstance(ch, splatlog.RichHandler)
assert ch.level == splatlog.DEBUG
LOG.info(
    "levels",
    root_handler=logging.getLevelName(logging.getLogger().level),
    console_handler=ch.get_level_name(),
)

INFO      __main__                                                                                                 
msg       levels                                                                                                   
data      root_handler    str          INFO                                                                        
          console_handler str          DEBUG

This will work, but it's silly.

In [7]:
splatlog.setup(level="notset", console="info")

LOG.info("printed")
LOG.debug("dropped")

INFO      __main__                                                                                                 
msg       printed

Put everything back the way it started, should be falling back to
`logging.lastResort` again.

In [8]:
splatlog.setup(level="WARNING", console=False)
LOG.warning("hey ya")

hey ya


Custom Named Handler
------------------------------------------------------------------------------

Example of adding a custom handler, confusingly named `custom`.

All I need to do is use the `@splatlog.named_handler` decorator to register a
builder function. The function will receive `Any` value that is given in
`splatlog.setup(custom=value)`, with the exception of `setup(custom=None)`,
which is ignored (no-op), and `setup(custom=False)`, which will remove the
`custom` handler without involving the builder.

The builder needs to return a `logging.Handler` instance, or raise an
`Exception`. Like the built-in named handlers, our builder accepts a variety of
values.

In [9]:
@splatlog.named_handler("custom", on_conflict="replace")
def to_custom_handler(value: Any) -> logging.Handler:
    LOG.debug("casting custom handler...", value=value)

    if value is True:
        return rich.logging.RichHandler()

    if isinstance(value, logging.Handler):
        return value

    if isinstance(value, Mapping):
        return rich.logging.RichHandler(**value)

    if splatlog.is_level(value):
        return rich.logging.RichHandler(level=value)

    if isinstance(value, Console):
        return rich.logging.RichHandler(console=value)

    raise ValueError(f"unexpected value: {value!r}")

Now we can setup a `custom` handler same as we would `console` or `export`.

In [10]:
splatlog.setup(level="info", custom=True)

LOG.info("custom handler with `rich.logging.RichHandler`")

[12/13/25 03:51:01] INFO     custom handler with `rich.logging.RichHandler`                         ]8;id=989716;file:///private/tmp/ipykernel_96886/2669929988.py\2669929988.py]8;;\:]8;id=320222;file:///private/tmp/ipykernel_96886/2669929988.py#3\3]8;;\

Clean-up by removing the `custom` handler.

In [11]:
splatlog.setup(custom=False)

LOG.info("should not see this")

Export (JSON) Handler
------------------------------------------------------------------------------

Play-testing the `export` named handler.

In [12]:
class ExportIO:
    io: StringIO

    def __init__(self):
        self.io = StringIO()

    def clear(self):
        self.io.seek(0)
        self.io.truncate()

    def get(self, clear: bool = True) -> list[Any]:
        json_lines = self.io.getvalue()
        records = []
        for line in json_lines.splitlines():
            records.append(json.loads(line))
        if clear:
            self.clear()
        return records

    def last(self, clear: bool = True) -> Any:
        records = self.get(clear)
        return records[-1]


eio = ExportIO()
splatlog.setup(level="debug", console=True, export=eio.io)

In [13]:
LOG.info("hey", ho="let's go")
eio.last()

INFO      __main__                                                                                                 
msg       hey                                                                                                      
data      ho          str          let's go

{'t': '2025-12-13T03:51:01.191347-08:00',
 'level': 'INFO',
 'name': '__main__',
 'file': '/private/tmp/ipykernel_96886/1282194280.py',
 'line': 1,
 'msg': 'hey',
 'data': {'ho': "let's go"}}

### Rich Messages ###

I bumped into this feature when figuring out if I could get rid of the
`splatlog.lib.rich.theme.DEFAULT_CONSOLE` (we can) because it doesn't mesh well
with setting a default theme (something I'm actually using), and the only place
the default theme was being used was in `splatlog.lib.rich.capture_riches`,
which is used to test if `is_rich(logging.LogRecord.msg)`, and if so capture 
the rich rendering output to return.

In practice, this feature is used for styling log messages with [Rich Console Markup][], like:

[Rich Console Markup]: https://rich.readthedocs.io/en/latest/markup.html

In [14]:
LOG.info("[bold red]alert![/bold red] Something happened")
eio.last()

INFO      __main__                                                                                                 
msg       alert! Something happened

{'t': '2025-12-13T03:51:01.195636-08:00',
 'level': 'INFO',
 'name': '__main__',
 'file': '/private/tmp/ipykernel_96886/3270632472.py',
 'line': 1,
 'msg': '[bold red]alert![/bold red] Something happened'}

What's interesting is that it doesn't even seem to _work_, as far as carrying the styling information into the JSON log.

Ok, that's not as terrible, but I'm still questionable about using the ANSI coded string.

Rich has other options, like exporting HTML:

In [15]:
rich_msg = Text.from_markup("[bold red]alert![/bold red] Something happened")

f_rec = StringIO()
c_rec = Console(
    # Need this, as otherwise the Jupyter detection will result in `file=` not
    # working (WTF..?)
    force_jupyter=False,
    # Where the console should write to
    file=f_rec,
    # Force terminal control codes
    force_terminal=True,
    # Boolean to enable recording of terminal output
    record=True,
)

# Why does this print to output?
c_rec.print(rich_msg)
html = c_rec.export_html(inline_styles=True)

m = re.search(r"(?is)<body[^>]*>(.*?)</body\s*>", html)
body = m.group(1) if m else ""
print(body)
display(HTML(body))


    <pre style="font-family:Menlo,'DejaVu Sans Mono',consolas,'Courier New',monospace"><code style="font-family:inherit"><span style="color: #800000; text-decoration-color: #800000; font-weight: bold">alert!</span> Something happened
</code></pre>



In [16]:
msg = "[blue]Starting[/blue] server on {host}:{port}"
msg.format(host="127.0.0.1", port=1234)

'[blue]Starting[/blue] server on 127.0.0.1:1234'

In [17]:
LOG.info(msg, host="127.0.0.1", port=1234)

fmt = splatlog.rich.RichFormatter()
fmt.format(msg, host="127.0.0.1", port=1234)

INFO      __main__                                                                                                 
msg       Starting server on 127.0.0.1:1234                                                                        
data      host        str          127.0.0.1                                                                       
          port        int          1234

Starting server on '127.0.0.1':1234